# Notebook to filter Nashville GeoJSON file

The neighborhood information provided by Zillow provides a lot of different neighborhoods.  In fact, it even includes some neightborhoods outside the Davidson county area. Currently, we have been parsing through the relevant boundaries manually in Javascript in the HTML file, but in this notebook we'll create a new geoJSON file only containing the neighborhoods we wish to display.

In [1]:
import pygeoj
import pandas as pd

In [2]:
neighborhoods = pygeoj.load(filepath="Zillow_Neighborhoods.geojson")
davidson = pd.read_csv("davidson_counties.csv", dtype=str, usecols=['RegionID'])

In [4]:
print("Our original geoJSON file has %s neighborhoods." %len(neighborhoods))
print("Our Davidson County file contains %s neighborhoods." %len(davidson))

Our original geoJSON file has 798 neighborhoods.
Our Davidson County file contains 662 neighborhoods.


In [5]:
filtered_json = pygeoj.new()
for feature in neighborhoods:
    if(davidson['RegionID'].str.contains(feature.properties['RegionID']).any()):
         filtered_json.add_feature(feature)

In [6]:
print("Our new file will have information for %s neighborhoods." % len(filtered_json))

Our new file will have information for 662 neighborhoods.


Run this code to add the z_indexes

In [7]:
import json
from pprint import pprint

In [8]:
with open('nHoodData.json') as f:
    data = json.load(f)

# pprint(data)

nhoods_w_zest = {}

# create map holding regionid as key and zindex as value. if no zindex, default to 0.
for i in range(len(data)):
    if 'zindex' in data[i]:
         nhoods_w_zest[data[i]['id']] = data[i]['zindex']
    else:
         nhoods_w_zest[data[i]['id']] = 0

In [9]:
# add property to all geojson
for i in range(len(filtered_json)):
    id = int(filtered_json[i].properties['RegionID'])
    if id in nhoods_w_zest:
        filtered_json[i].properties['zindex'] = nhoods_w_zest[id]
    else:
        filtered_json[i].properties['zindex'] = 0
        


In [14]:
for i in range(len(filtered_json)):
    print(filtered_json[i].properties)

{'State': 'TN', 'County': 'Davidson', 'City': 'Nashville', 'Name': 'Cloverhill', 'RegionID': '225896', 'zindex': 123400}
{'State': 'TN', 'County': 'Davidson', 'City': 'Nashville', 'Name': 'Germantown', 'RegionID': '226807', 'zindex': 292500}
{'State': 'TN', 'County': 'Davidson', 'City': 'Nashville', 'Name': 'Scottsboro', 'RegionID': '229588', 'zindex': 0}
{'State': 'TN', 'County': 'Davidson', 'City': 'Nashville', 'Name': 'Sherwood Forest', 'RegionID': '229697', 'zindex': 0}
{'State': 'TN', 'County': 'Davidson', 'City': 'Nashville', 'Name': 'Bakers Grove', 'RegionID': '272842', 'zindex': 0}
{'State': 'TN', 'County': 'Davidson', 'City': 'Nashville', 'Name': 'Bells Bend', 'RegionID': '272926', 'zindex': 0}
{'State': 'TN', 'County': 'Davidson', 'City': 'Nashville', 'Name': 'Buena Vista Heights', 'RegionID': '273085', 'zindex': 72100}
{'State': 'TN', 'County': 'Davidson', 'City': 'Nashville', 'Name': 'Capitol View', 'RegionID': '273133', 'zindex': 0}
{'State': 'TN', 'County': 'Davidson', 'C

In [11]:
filtered_json.add_all_bboxes()
filtered_json.update_bbox()
filtered_json.add_unique_id()
filtered_json.save("Zillow_w_index.geojson")

In [10]:
# make csv file only with nonzero zindexes

with open('nHoodData.json') as f:
    data = json.load(f)

# pprint(data)

nhoods_w_zest = {}

# create map holding regionid as key and zindex as value. if no zindex, default to 0.
for i in range(len(data)):
    if 'zindex' in data[i]:
         nhoods_w_zest[data[i]['id']] = data[i]['zindex']
    else:
         nhoods_w_zest[data[i]['id']] = 0
            
# add property to all geojson
for i in range(len(filtered_json)):
    id = int(filtered_json[i].properties['RegionID'])
    if id in nhoods_w_zest:
        filtered_json[i].properties['zindex'] = nhoods_w_zest[id]
    else:
        filtered_json[i].properties['zindex'] = 0
        
filtered_sin_0 = pygeoj.new()

for i in range(len(filtered_json)):
    if(filtered_json[i].properties['zindex'] != 0):
        filtered_sin_0.add_feature(filtered_json[i])

In [20]:
filtered_sin_0.add_all_bboxes()
filtered_sin_0.update_bbox()
# filtered_sin_0.add_unique_id()
filtered_sin_0.save("Zillow_zindexes_only.geojson")

In [17]:
import pandas as pd

65


In [16]:
# code to write existing (with zindex) neighborhoods to a csv to check Zillow for rentals
df = pd.DataFrame()
indexes = []
ids = []
names = []
for i in range(len(filtered_sin_0)):
    names.append(filtered_sin_0[i].properties['Name'])
    indexes.append(filtered_sin_0[i].properties['zindex'])
    ids.append(filtered_sin_0[i].properties['RegionID'])
    
df['Name'] = names
df['RegionID'] = ids
df['zindex'] = indexes

df.to_csv('Zillow_Neighborhoods_names.csv')
